In [13]:
import matplotlib.pylab as plt
%matplotlib inline
import numpy as np
from numpy.random import RandomState
from itertools import product

# dane
from sklearn.datasets import fetch_mldata
# splity
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
# modele
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
# metryki
from sklearn.metrics import accuracy_score

mnist = fetch_mldata('MNIST original')
X = mnist["data"]
y = mnist["target"]

# zawsze przed uczeniem/splitami proszę zrobić shuffle na danych!

In [2]:
# sampler pamięta jeden konkretny rozkład prawdopodobieństwa
# jedynym argumentem samplera jest obiekt numpy.random.RandomState, który oznaczamy rng
# sampler zwraca jedną wartość ze swojego rozkładu
# ponieważ rng pamięta swój stan, to możemy np. raz stworzyć go na początku i podawać w pętli

# rodzina rozkładów jednostajnych na podanych listach elementów
# sampler losuje z rozkładu jednostajnego na liście l
def uniform_from_list(l):
    def sampler(rng):
        return l[rng.randint(0,len(l))]
    return sampler

# rodzina rozkładów jednostajnych na przedziałach liczb całkowitych
# sampler losuje z rozkładu jednostajnego na podzbiorze liczb całkowitych od low (włącznie) do high (wyłącznie)
def uniform_int_on_interval(low, high):
    def sampler(rng):
        return rng.randint(low,high)
    return sampler

# rodzina rozkładów jednostajnych na przedziałach
# sampler losuje z rozkładu jednostajnego na przedziale [low, high]
def uniform_on_interval(low, high):
    def sampler(rng):
        return rng.uniform(low, high)
    return sampler

# rodzina rozkładów jednostajnych na przedziałach w wykładniku potęgi liczby 10
# sampler losuje liczbę alpha z rozkładu jednostajnego na przedziale [low, high], a następnie zwraca 10^alpha
def log_uniform_on_interval(low, high):
    def sampler(rng):
        return 10.**rng.uniform(low, high)
    return sampler

# rodzina rozkładów gaussa
# sampler losuje liczbę z rozkładu N(mean, std^2)
def normal(mean, std):
    def sampler(rng):
        return rng.normal(loc=mean, scale=std)
    return sampler

# rodzina rozkładów gaussa w wykładniku potęgi liczby 10
# sampler losuje liczbę alpha z rozkładu N(mean, std^2), a następnie zwraca 10^alpha
def log_normal(mean, std):
    def sampler(rng):
        return 10**rng.normal(loc=mean, scale=std)
    return sampler

In [3]:
def grid_search(grid):
    (keys, values_grid) = zip(*grid.iteritems())
    
    for values in product(*values_grid):
        yield dict(zip(keys, values))

def random_grid_search(grid, k=20, random_state=43):
    (keys, values_grid) = zip(*grid.iteritems())
    rng = RandomState(random_state) # ustalamy jeden wspólny rng
    l=1
    for i in grid.values() :
        l=l*len(i)
    print l
    list=rng.choice(l,k,replace=False)
    P=product(*values_grid)
    print list
    cross=[x for x in P]
    random_cross=[cross[i] for i in list]
    print random_cross
    for values in random_cross:
        yield dict(zip(keys, values))
    # wysamplować k zestawów hiperparametrów
    # ...
    #yield dict_of_hyperparams
random_grid_search({'C': [0.1, 1., 10., 100.], 'gamma': [0.0001, 0.0003, 0.001]},k=2)
def random_search(grid, k=20, random_state=43):
    rng = RandomState(random_state) # ustalamy jeden wspólny rng
    (keys, samplers) = zip(*sorted(grid.iteritems())) # sortujemy klucze, kolejność samplowania jest ważna!
    print  samplers
    random_cross=[tuple([sampler(rng) for sampler in samplers]) for i in range(0,k)]
    print random_cross
    for values in random_cross:
        print values
        yield dict(zip(keys, values))
 

In [136]:
def double_skf_model_evaluation2(Model, generator_function, generator_function_kwargs, X, y, metric, selection_n_splits, evaluation_n_splits, random_state):
    """
    Model - klasa modelu
    generator_function, generator_function_kwargs - chcemy zrobić coś w stylu:
        for hyperparams in generator_function(**generator_function_kwargs):
            ...
        nie podajemy wprost generatora, bo trzeba go użyć wielokrotnie, a generatorów (chyba?) nie da się kopiować
    X,y - dane i etykiety
    metric - funkcja o sygnaturze metric(y_true, y_pred), która ocenia skuteczność nauczonego modelu
    selection/evaluation_n_splits - liczba splitów/foldów w odpowiednim cross validation
    random_state - używany wszędzie tam, gdzie trzeba
    """
    hyparlist = [ hyperparams  for hyperparams in generator_function(generator_function_kwargs)]
  
    
    score=0   
    # 1. Dzielimy evaluation_n_splits razy (X,y) na (X_train,y_train), (X_test,y_test).
    skf_1 = StratifiedKFold(n_splits=evaluation_n_splits, random_state=random_state, shuffle=True)
    # 2. Dla każdego takiego podziału:
    for index_train1 , index_test in skf_1.split(X,y):
        skf_2=StratifiedKFold(n_splits=selection_n_splits, random_state=random_state, shuffle=True)
        X_1=X[index_train1]
        y_1=y[index_train1]
        for index_train2 , index_valid in skf_1.split(X_1,y_1):
            Model_tab=[]
            for hipar in hyparlist:
                Model_tab=Model_tab+[Model(**hipar)]
            #model_score_list=[0. for model in Model_tab]
            for model in Model_tab:
                model.fit(X_1[index_train2],y_1[index_train2])
                score = metric(y[index_valid], Model_tab[0].predict(X[index_valid]))
            model_score_list=np.array([metric(y_1[index_valid], model.predict(X_1[index_valid])) for model in Model_tab])
        best_hypar = hyparlist[np.argmax(model_score_list)]
        model = Model(**best_hypar)
        model.fit(X_1, y_1)
        score =score+ metric(y[index_test], model.predict(X[index_test]))
        # 1. Dzielimy selection_n_splits razy (X_train,y_train) na (X_train2,y_train2), (X_valid,y_valid).
        # 2. Dla każdego zestawu hiperparametrów:
            # 1. Dla każdego podziału (X_train2,y_train2), (X_valid,y_valid):
                # 1. Tworzymy model = Model(zestaw_hiperparametrów).
                # 2. Uczymy model na (X_train2,y_train2), testujemy na (X_valid,y_valid) i otrzymujemy score.
                # 3. (zestaw_hiperparametrów, score) zapisujemy w tabelce.
        # 3. Dla każdego zestawu hiperparametrów mamy trzy różne score z trzech podziałów - uśredniamy je.
        # 4. Wybieramy średnio najlepszy zestaw hiperparametrów.
        # 5. Tworzymy model = Model(średnio_najlepszy_zestaw_hiperparametrów) i uczymy na całym (X_train,y_train).
        # 6. Testujemy model na (X_test,y_test) i otrzymujemy evaluation_score, zapisujemy go.
    # 3. Uśredniamy trzy evaluation_score, zwracamy średnią jako ostateczny score Modelu.
    return score/evaluation_n_splits

zad 4

In [138]:



idx = np.random.RandomState(743).permutation(len(y))
_X = X[idx[:5000]]
_y = y[idx[:5000]]

x=double_skf_model_evaluation2(
    Model=KNeighborsClassifier,
    generator_function=grid_search,
    generator_function_kwargs={'n_neighbors':[4,11], 'leaf_size' : [20,30,40]},
    X=_X, y=_y,
    metric=accuracy_score,
    selection_n_splits=2, 
    evaluation_n_splits=2,
    random_state=43)
print "Wynik: ",x


{'n_neighbors': 4, 'leaf_size': 20}
Wynik:  0.908326661329


Problemem jest to, że jeśli przeprowadzimy augmentacje na całych danych a potem wszysko podzielimy na zbiór uczący i testujący to może się okazać, że w zbiorze testującym będą przykłady takie same jak w uczącym tylko z dodanym szumem. Co powoduje, że testowanie na nich zaburza wynik trafności predykcji modelu. W modelu K=1 sąsiadów etykieta dla obrazka testowanego jest brana z najbliższego z obrazków w zbiorze testowym a jeśli jest tam ten sam przykład ale z innym szumym do prawie na 100 % jego etykieta zostanie wybrana

In [5]:
#Według mnie tak powina być przeprowadzona prawidłowa augmentacja danych
X = mnist["data"]
y = mnist["target"]
idx = np.random.permutation(len(y))
X = X[idx[:3000]]
y = y[idx[:3000]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=43)
for i in xrange(7):
    np.append(X_train,X.copy() + np.random.normal(scale=3, size=X.shape))
    np.append(y_train,y.copy())
model = KNeighborsClassifier(n_neighbors=1)
print "fitting KNN (with augmentation)"
model.fit(X_train, y_train)
print "train accuracy:", accuracy_score(y_train, model.predict(X_train))
print "test accuracy:", accuracy_score(y_test, model.predict(X_test))

fitting KNN (with augmentation)
train accuracy: 1.0
test accuracy: 0.913131313131
